In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3436460/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_1_5_1,0.964920,0.820862,0.924504,0.849178,0.038824,0.133603,0.022732,0.081428,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
1,model_1_5_0,0.963728,0.820860,0.971342,0.861469,0.040142,0.133605,0.008629,0.074792,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
2,model_1_5_2,0.963630,0.816152,0.841932,0.824063,0.040251,0.137116,0.047594,0.094987,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
3,model_1_5_3,0.961125,0.809250,0.746295,0.793913,0.043023,0.142263,0.076391,0.111265,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
4,model_1_5_4,0.957850,0.802310,0.640689,0.761123,0.046648,0.147439,0.108189,0.128968,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
...,...,...,...,...,...,...,...,...,...,...
148,model_2_5_15,0.902869,0.788859,0.884427,0.860976,0.107495,0.151828,0.118077,0.135945,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
150,model_2_8_0,0.902347,0.716273,0.971608,0.926387,0.108073,0.063057,0.009271,0.037746,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
152,model_1_9_1,0.900868,0.656229,0.898586,0.798853,0.109710,0.347712,0.153359,0.256254,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
159,model_1_7_3,0.898835,0.820998,0.813658,0.829799,0.111960,0.218356,0.141701,0.182283,"Hidden Size=[40, 20], regularizer=0.1, learnin..."
